# Ridge Regression

# 1. Preprocessing Data

In [130]:
# Import the libraries
import pandas as pd 
import numpy as np
import re 

In [131]:
# Processing data txt 
with open("/Users/charles/MLGT/SESSION 1/Data/DeathRate.txt") as f:
    # Read data, split each line, store in list
    raw_data_lines = f.read().splitlines()
    lines = []
    for i in range(len(raw_data_lines)):
        # Delete some space in each line with regex (raw)
        line = re.sub(" +", " ", raw_data_lines[i])
        # Delete first row
        line = line.split()[1:]
        lines.append(line)

In [132]:
# Assign info for first row
init_col = np.append(["A" + str(i) for i in range(1,16)], "B")
# Covert to DF 
data = pd.DataFrame(lines, columns=init_col)
# Change index (first col)
data.index = [i for i in range(1, len(data)+1)]

In [133]:
# Convert data to numeric 
data = data.apply(pd.to_numeric)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 1 to 60
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      60 non-null     int64  
 1   A2      60 non-null     int64  
 2   A3      60 non-null     int64  
 3   A4      60 non-null     float64
 4   A5      60 non-null     float64
 5   A6      60 non-null     float64
 6   A7      60 non-null     float64
 7   A8      60 non-null     int64  
 8   A9      60 non-null     float64
 9   A10     60 non-null     float64
 10  A11     60 non-null     float64
 11  A12     60 non-null     int64  
 12  A13     60 non-null     int64  
 13  A14     60 non-null     int64  
 14  A15     60 non-null     int64  
 15  B       60 non-null     float64
dtypes: float64(8), int64(8)
memory usage: 8.0 KB


In [134]:
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,B
1,36,27,71,8.1,3.34,11.4,81.5,3243,8.8,42.6,11.7,21,15,59,59,921.870
2,35,23,72,11.1,3.14,11.0,78.8,4281,3.6,50.7,14.4,8,10,39,57,997.875
3,44,29,74,10.4,3.21,9.8,81.6,4260,0.8,39.4,12.4,6,6,33,54,962.354
4,47,45,79,6.5,3.41,11.1,77.5,3125,27.1,50.2,20.6,18,8,24,56,982.291
5,43,35,77,7.6,3.44,9.6,84.6,6441,24.4,43.7,14.3,43,38,206,55,1071.289


# 2. Normalize Data

In [149]:
def normalize_and_add_ones(X):
    X = np.array(X)
    # Normalize MinMaxScaler (0,1), search in each row to find min/max element in col
    X_min = np.array([[np.amin(X[:, id_col]) 
                        for id_col in range(X.shape[1])]     # Check all col in each row
                            for _ in range(X.shape[0])])     # Loop all rows
    X_max = np.array([[np.amax(X[:, id_col]) 
                        for id_col in range(X.shape[1])]     # Check all col in each row
                            for _ in range(X.shape[0])])     # Loop all rows
    X_normalized = (X - X_min) / (X_max - X_min)

    # X_bias with 1 for first column
    X_bias = np.concatenate((np.ones((X_normalized.shape[0],1)), X_normalized), axis = 1)
    return X_bias
normalize_and_add_ones(data)

array([[1.        , 0.52      , 0.21126761, ..., 0.20938628, 0.6       ,
        0.40672347],
       [1.        , 0.5       , 0.15492958, ..., 0.13718412, 0.54285714,
        0.64245417],
       [1.        , 0.68      , 0.23943662, ..., 0.11552347, 0.45714286,
        0.53228523],
       ...,
       [1.        , 0.7       , 0.16901408, ..., 0.02527076, 0.51428571,
        0.32554439],
       [1.        , 0.64      , 1.        , ..., 0.1732852 , 0.45714286,
        0.37554393],
       [1.        , 0.56      , 0.22535211, ..., 0.13718412, 0.57142857,
        0.50774604]])

IndexError: tuple index out of range

# 3. Implement class Ridge Regression

![](2022-02-25-22-34-21.png)